In [1]:
!pip install -r requirements.txt

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-kmd9qu1v
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-kmd9qu1v
  Resolved https://github.com/huggingface/transformers to commit 312b104ff65514736c0475814fec19e47425b0b5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached datasets-2.12.0-py3-none-any.whl (474 kB)
  Using cached jsonlines-3.1.0-py3-none-any.whl (8.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 622.4 kB/s eta 0:00:00a 0:00:01
  Using cached accelerate-0.18.0-py3-none-any.whl (215 kB)
  Using cached bitsandbytes-0.38.1-py3-none-any.whl (104.3 MB)
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
  Using cached tokenizers-0.13.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached fsspec-2023.4.0-py3-none-any.whl (153 kB)
 

In [1]:
import torch
import numpy as np

torch.cuda.device_count()

1

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("hf-internal-testing/llama-tokenizer")

In [3]:
from datasets import load_dataset, load_from_disk

dataset = load_from_disk("./chunk_dataset")
val_dataset = load_dataset("IlyaGusev/rulm", split="validation[:10%]")

No config specified, defaulting to: rulm/default
Found cached dataset rulm (/home/jovyan/.cache/huggingface/datasets/IlyaGusev___rulm/default/0.0.1/c94da05286e8839c1219f9a5061c630709a487fb298427a6f5a73ee0fe2cd2cb)


In [4]:
len(dataset[0]['input_ids'])

512

In [5]:
def tokenize(prompt, add_eos_token=True):
    result = tokenizer(prompt)
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    return result

In [6]:
from itertools import chain
def chunk_examples(examples, max_length):
    input_ids_chunks = []
    attention_mask_chunks = []
    # print(len(examples["input_ids"]))
    input_ids = list(chain(*examples["input_ids"]))
    attention_mask = list(chain(*examples["attention_mask"]))
    input_ids_chunks.extend(
        [
            input_ids[i : i + max_length]
            for i in range(0, len(input_ids), max_length)
        ]
    )
    attention_mask_chunks.extend(
        [
            attention_mask[i : i + max_length]
            for i in range(0, len(attention_mask), max_length)
        ]
    )
    return {
        "input_ids": input_ids_chunks,
        "attention_mask": attention_mask_chunks,
    }

In [7]:
def prepare_dataset(dataset):
    tokenized_dataset = dataset.map(lambda x: tokenize(x['text']),
                                    num_proc=16, remove_columns=["meta"])
    chunk_dataset = tokenized_dataset.map(lambda x: chunk_examples(x, 512), batched=True, batch_size=100,
                                          num_proc=16, remove_columns=tokenized_dataset.column_names)
    return tokenized_dataset, chunk_dataset

In [8]:
_, chunk_val_dataset = prepare_dataset(val_dataset)

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/IlyaGusev___rulm/default/0.0.1/c94da05286e8839c1219f9a5061c630709a487fb298427a6f5a73ee0fe2cd2cb/cache-f418d707dbed0c01_*_of_00016.arrow
Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/IlyaGusev___rulm/default/0.0.1/c94da05286e8839c1219f9a5061c630709a487fb298427a6f5a73ee0fe2cd2cb/cache-826a7ac955c9b816_*_of_00016.arrow


In [9]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.unk_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, return_tensors="pt")

In [10]:
from peft import get_peft_model, LoraConfig, prepare_model_for_int8_training

# model = AutoModelForCausalLM.from_pretrained("decapoda-research/llama-7b-hf")
model = AutoModelForCausalLM.from_pretrained("decapoda-research/llama-7b-hf", load_in_8bit=True, device_map='auto')


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/jovyan/.imgenv-heuristic-hypatia-0/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/jovyan/.imgenv-heuristic-hypatia-0/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/jovyan/.imgenv-heuristic-hypatia-0/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /home/jovyan/.imgenv-heuristic-hypatia-0 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/jovyan/.imgenv-heuristic-hypatia-0/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/home/jovyan/.imgenv-heuristic-hypatia-0/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/jovyan/.imgenv-heuristic-hypatia-0/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: T

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [11]:
lc = {
        "r": 8,
        "lora_alpha": 16,
        "lora_dropout": 0.05,
        "bias": "none",
        "target_modules": ["q_proj", "v_proj"],
        "task_type": "CAUSAL_LM"
    }

lora_config = LoraConfig(**lc)
model = prepare_model_for_int8_training(model, use_gradient_checkpointing=False)
model = get_peft_model(model, lora_config)

In [14]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): Dropout(p=0.05, inplace=False)
                (lora_A): Linear(in_features=4096, out_features=8, bias=False)
                (lora_B): Linear(in_features=8, out_features=4096, bias=False)
              )
              (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
              (v_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): Dropout(p=0.05, inplace=False)
                (lora_A): Linear(in_features=4096, out_features=8, bias=False)
                (lora_B): Linear(in_features=

In [12]:
# "trainer": {
#     "evaluation_strategy": "steps",
#     "per_device_train_batch_size": 16,
#     "per_device_eval_batch_size": 16,
#     "gradient_accumulation_steps": 8,
#     "eval_steps": 75,
#     "save_steps": 75,
#     "logging_steps": 5,
#     "learning_rate": 0.0003,
#     "num_train_epochs": 3,
#     "lr_scheduler_type": "cosine",
#     "warmup_steps": 50,
#     "fp16": true,
#     "bf16": false,
#     "torch_compile": false,
#     "optim": "adamw_torch"
# },
training_args = TrainingArguments(
    evaluation_strategy="steps",
    save_steps=75,
    eval_steps=75,
    warmup_steps=50,
    logging_steps=5,

    gradient_accumulation_steps=16,

    num_train_epochs=1,

    learning_rate=0.0003,
    lr_scheduler_type="cosine",

    per_device_eval_batch_size=4,
    per_device_train_batch_size=4,
    
    gradient_checkpointing=False,

    fp16=True,
    # bf16=False,
    torch_compile=False,
    optim="adamw_torch",

    logging_dir='./logs',
    output_dir='./trained',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=chunk_val_dataset,
    data_collator=data_collator
)

In [13]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [42]:
torch.cuda.empty_cache()

In [29]:
import torch

In [16]:
data = None

In [43]:
data = data_collator([dataset[0], dataset[1]])['input_ids'].to('cuda')

In [44]:
generate_ids = cuda_m(data)

In [64]:
generate_ids.logits.grad

In [45]:
type(generate_ids)

transformers.modeling_outputs.CausalLMOutputWithPast

In [65]:
del generate_ids

In [41]:
del data

In [23]:
generate_ids.keys()

odict_keys(['logits', 'past_key_values'])